# 0. Setup

Importing required packages and storing functions


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re

# 1. Data Preparation

Loading raw data and preparing them in a format that can be used for analysis

## 1.1 COVID vaccination Data

In [3]:
covid_vaccination = pd.read_csv('raw_data/COVID-19_Vaccinations_in_the_United_States_County.csv')
covid_vaccination.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,11/30/2022,49035,48,Salt Lake County,UT,97.7,943736,81.3,932131.0,86.6,...,4.0,4.0,4.0,4.0,1160437,1076750.0,225458.0,955797,851292,130051.0
1,11/23/2022,49035,47,Salt Lake County,UT,97.7,943646,81.3,932042.0,86.6,...,4.0,4.0,4.0,4.0,1160437,1076750.0,225458.0,955797,851292,130051.0
2,11/16/2022,49035,46,Salt Lake County,UT,97.7,940864,81.1,930040.0,86.4,...,4.0,4.0,4.0,4.0,1160437,1076750.0,225458.0,955797,851292,130051.0
3,11/09/2022,49035,45,Salt Lake County,UT,97.7,940731,81.1,929907.0,86.4,...,4.0,4.0,4.0,4.0,1160437,1076750.0,225458.0,955797,851292,130051.0
4,11/02/2022,49035,44,Salt Lake County,UT,97.7,940510,81.0,929686.0,86.3,...,4.0,4.0,4.0,4.0,1160437,1076750.0,225458.0,955797,851292,130051.0


In [8]:
# Convert date to datetime
covid_vaccination['Date'] = pd.to_datetime(covid_vaccination['Date'])
# Selecting columns for analysis
useful_columns = ['Date', 
                'Administered_Dose1_Pop_Pct', 
                'Series_Complete_Pop_Pct', 
                'Booster_Doses_Vax_Pct']

covid_vaccination = covid_vaccination[useful_columns]

In [10]:
covid_vaccination.describe()

,Administered_Dose1_Pop_Pct,Series_Complete_Pop_Pct,Booster_Doses_Vax_Pct
count,575.000000,575.000000,208.000000
mean,54.193043,46.325739,46.255288
std,26.118086,24.756166,6.737004
min,0.000000,0.000000,28.200000
25%,39.800000,28.250000,43.525000
50%,64.800000,56.200000,49.000000
75%,77.200000,68.950000,50.500000
max,81.300000,72.300000,54.900000


## 1.2 Census Income Data

In [17]:
census_files = glob.glob('raw_data/*5Y*.csv') + glob.glob('raw_data/*1Y2021*.csv')
census_files

['raw_data/ACSST5Y2020.S1903-2022-12-02T001609.csv',
 'raw_data/ACSST5Y2018.S1903-2022-12-02T001748.csv',
 'raw_data/ACSST5Y2019.S1903-2022-12-02T001721.csv',
 'raw_data/ACSST1Y2021.S1903-2022-12-02T001544.csv']

In [125]:
def get_census_data(files):
    age_df = pd.DataFrame(columns=["Label (Grouping)", "Median Income", "Margin of error", "Year"])
    race_df = pd.DataFrame(columns=["Label (Grouping)", "Median Income", "Margin of error", "Year"])
    for file in files:
        df = pd.read_csv(file).iloc[0:17, [0,-2,-1]]
        year = re.search(r'\d{4}', file).group()
        df["Year"] = year
        df["Year"] = pd.to_datetime(df["Year"])
        df.rename(columns={df.columns[-2]: "Margin of error", df.columns[1]: "Median Income"}, inplace=True)
        df["Margin of error"] = df["Margin of error"].str.replace("±|,", "", regex=True).str.strip().astype(float)
        df["Median Income"] = df["Median Income"].str.replace(",", "").str.strip().astype(float)
        df["Lower Bound"] = df["Median Income"] - df["Margin of error"]
        df["Upper Bound"] = df["Median Income"] + df["Margin of error"]
        race = df.iloc[3:9,:]
        race_df = pd.concat([race_df, race])
        age = df.iloc[13:17,:]
        age_df = pd.concat([age_df, age])

    return race_df.sort_values(by=["Label (Grouping)", "Year"]).reset_index(drop=True),\
             age_df.sort_values(by=["Label (Grouping)", "Year"]).reset_index(drop=True)

get_census_data(census_files)[1]

,Label (Grouping),Median Income,Margin of error,Year,Lower Bound,Upper Bound
0,15 to 24 years,41834.0,2041.0,2018-01-01,39793.0,43875.0
1,15 to 24 years,43096.0,1797.0,2019-01-01,41299.0,44893.0
2,15 to 24 years,45604.0,2231.0,2020-01-01,43373.0,47835.0
3,15 to 24 years,46098.0,2813.0,2021-01-01,43285.0,48911.0
4,25 to 44 years,74241.0,898.0,2018-01-01,73343.0,75139.0
5,25 to 44 years,78225.0,1528.0,2019-01-01,76697.0,79753.0
6,25 to 44 years,81472.0,1396.0,2020-01-01,80076.0,82868.0
7,25 to 44 years,86988.0,2410.0,2021-01-01,84578.0,89398.0
8,45 to 64 years,85766.0,1633.0,2018-01-01,84133.0,87399.0
9,45 to 64 years,90863.0,1463.0,2019-01-01,89400.0,92326.0


In [64]:
race_income_df, age_income_df = get_census_data(census_files)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 3 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Label (Grouping)  24 non-null     object 
 1   Median Income     24 non-null     float64
 2   Margin of error   24 non-null     float64
 3   Year              24 non-null     object 
dtypes: float64(2), object(2)
memory usage: 960.0+ bytes


## 1.3 BLS Employment Data

In [90]:
employment_files = glob.glob('raw_data/*.xlsx')
employment_files

['raw_data/SeriesReport-20221201195433_359946.xlsx',
 'raw_data/SeriesReport-20221201195525_253beb.xlsx',
 'raw_data/SeriesReport-20221201195509_7a60a6.xlsx',
 'raw_data/SeriesReport-20221201195335_720302.xlsx',
 'raw_data/SeriesReport-20221201195501_56ace2.xlsx',
 'raw_data/SeriesReport-20221201195025_e1cf6d.xlsx',
 'raw_data/SeriesReport-20221201195517_ea1977.xlsx',
 'raw_data/SeriesReport-20221201194950_be8d0c.xlsx',
 'raw_data/SeriesReport-20221201195451_2273d7.xlsx',
 'raw_data/SeriesReport-20221201195414_8fce05.xlsx',
 'raw_data/SeriesReport-20221201195015_915064.xlsx']

In [113]:
def get_employment_data(files):
    employment_df = pd.DataFrame(columns=["Employment", "Date", "Sector"])
    for file in files:
        series = pd.read_excel(file, skiprows=12, engine='openpyxl')
        sector = pd.read_excel(file, skiprows=7, engine='openpyxl').iloc[0,1]
        series = pd.melt(series.iloc[:,:-1], id_vars=['Year'], var_name='Month', value_name='Employment')
        series['Date'] = pd.to_datetime(series['Year'].astype(str) + '-' + series['Month'].astype(str))
        series['Employment'] = series['Employment']*1000
        series['Sector'] = sector
        series = series.drop(columns=['Year', 'Month'])
        series.sort_values(by='Date', inplace=True)
        employment_df = pd.concat([employment_df, series])
        employment_df.reset_index(inplace=True, drop=True)
    
    return employment_df

import warnings
with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    employment_df = get_employment_data(employment_files)

In [127]:
employment_df.head()

,Employment,Date,Sector
0,58400.0,2019-01-01,Financial Activities
1,58600.0,2019-02-01,Financial Activities
2,58700.0,2019-03-01,Financial Activities
3,59200.0,2019-04-01,Financial Activities
4,59800.0,2019-05-01,Financial Activities


# 3. Analysis

## 3.1 Research Question 1

Which industries and occupations suffered the most employment cuts and which ones witnessed a boom in employment because of the pandemic?

## 3.2 Research Question 2

How have income and employment growth patterns changed post-pandemic?

## 3.3 Research Question 3

Do vaccination rates have a part to play in affecting employment patterns?

## 3.4 Research question 4

Have vaccinations affected growth of COVID cases and COVID mortality? Were booster shots impactful in containing the pandemic?